In [ ]:
import contextily
import dask_geopandas
import deltalake
import geopandas as gpd
import mercantile
import pandas as pd
import pickle
import planetary_computer as pc
import pystac_client
import shapely.geometry
from shapely.geometry import box

import os
os.chdir(r'C:\code\bedford-ubid')

from pymodule.file_ops import read_pickle, write_pickle

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:


# bounds for bedford
bounds = [-73.74815136, 41.16565413, -73.59163084, 41.28549615]

# Connect to Planetary Computer STAC API
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=pc.sign_inplace
)

# Get signed asset
search = catalog.search(collections=["ms-buildings"], bbox=bounds)
item = next(search.items())
signed_item = pc.sign(item)
asset = signed_item.assets['data']

# Create a bounding box geometry for spatial filtering
bbox_geom = box(*bounds)

In [3]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=pc.sign_inplace,
)

collection = catalog.get_collection("ms-buildings")
collection

<CollectionClient id=ms-buildings>

In [4]:
# Explore the collection structure
print("Collection assets:")
for akk, akv in collection.assets.items():
    print(f"- {akk}: {akv.href}")
    if hasattr(akv, 'extra_fields'):
        print(f"  Extra fields: {akv.extra_fields.keys()}")

# Look at the collection metadata
print("\nCollection extra fields:")
print(collection.extra_fields.keys())

# Check if there are spatial/temporal extents
print(f"\nSpatial extent: {collection.extent.spatial}")
print(f"Temporal extent: {collection.extent.temporal}")

Collection assets:
- delta: az://footprints/delta/2023-04-25/ml-buildings.parquet/
  Extra fields: dict_keys(['table:storage_options'])
- thumbnail: https://ai4edatasetspublicassets.blob.core.windows.net/assets/pc_thumbnails/msbuildings-thumbnail.png
  Extra fields: dict_keys([])
- global-footprints: https://planetarycomputer.microsoft.com/api/data/v1/vector/collections/ms-buildings/tilesets/global-footprints/tilejson.json
  Extra fields: dict_keys([])

Collection extra fields:
dict_keys(['type', 'item_assets', 'msft:region', 'table:columns', 'msft:container', 'msft:storage_account', 'msft:short_description'])

Spatial extent: <pystac.collection.SpatialExtent object at 0x00000184980E6A50>
Temporal extent: <pystac.collection.TemporalExtent object at 0x00000184980E6BA0>


In [5]:
quadkeys_of_interest = [
    30232332,
    30232333,
    32010110,
    32010111,
]

search = catalog.search(collections=["ms-buildings"])
items = search.items()

print("Available regions:")
items_found = []
for num, item in enumerate(items):
    ipop = item.properties
    if 'msbuildings:region' in ipop:
        region = item.properties['msbuildings:region'].lower()
        if 'united' in region:
            if 'msbuildings:quadkey' in ipop:
                quadkey = item.properties['msbuildings:quadkey']
                if quadkey in quadkeys_of_interest:
                    print(f"  Quadkey: {item.properties['msbuildings:quadkey']}")
                    items_found.append(ipop)


Available regions:
  Quadkey: 32010111
  Quadkey: 32010110
  Quadkey: 30232333
  Quadkey: 30232332


### actual query

In [6]:
quadkeys_of_interest = [
    30232332,
    30232333,
    32010110,
    32010111,
]

quadkey_items_found = []

for q in quadkeys_of_interest:
    search = catalog.search(
        collections=["ms-buildings"],
        query={"msbuildings:quadkey": {"eq": str(q)}}  # Example
    )
    items = list(search.items())
    for i in items:
        quadkey_items_found.append(i)


In [7]:
quadkey_results_df = []
for q in quadkey_items_found:
    result = dask_geopandas.read_parquet(
        q.assets['data'].href,
        storage_options=asset.extra_fields["table:storage_options"],
    )
    computed = result.compute()
    quadkey_results_df.append(computed)


In [8]:

result_df = pd.concat(quadkey_results_df)

str(result_df.iloc[9]['geometry'])
str(result_df.iloc[0].geometry)


'POLYGON ((-74.25526153715828 41.11706909559916, -74.25518939815252 41.11695571945191, -74.25509561306403 41.1169895880365, -74.25516775206978 41.117102964125245, -74.25526153715828 41.11706909559916))'

In [ ]:
with open('./dataprocess/planetary_data.pickle', 'wb') as f:
    pickle.dump(result_df, f)